In [36]:
import re
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text) #keep words/numbers/whitespace
    return text.split()

#test preprocessing
print(preprocess("Hello! How's the weather?"))

['hello', 'hows', 'the', 'weather']


In [ ]:
import re
import math
from collections import defaultdict
from preprocess import preprocess


def create_ngrams_dict():
    return defaultdict(int)

class NGramModel:
    def __init__(self, n=3):
        self.n = n
        self.ngrams = defaultdict(create_ngrams_dict)
        self.vocab = set()

    def train(self, corpus):
        #collect n-grams and vocabulary
        print(f"Training on {len(corpus)} sentences...")
        for sentence in corpus:
            tokens = preprocess(sentence)
            if not tokens:
                continue
            self.vocab.update(tokens)
            print(f"Processing sentence: {tokens}")  # Debug

            for i in range(len(tokens)-self.n+1):
                context = tuple(tokens[i:i+self.n-1])
                next_word = tokens[i+self.n-1]
                self.ngrams[context][next_word]+=1
                print(f"Added n-gram: {context} -> {next_word}")  # Debug

        #add laplace smoothing
        self._apply_smoothing()
        print(f"Vocabulary size: {len(self.vocab)}")
        print(f"Total contexts: {len(self.ngrams)}")

    def _apply_smoothing(self, alpha=0.1):
        vocab_size = len(self.vocab)
        for context in self.ngrams:
            total = sum(self.ngrams[context].values())+alpha*vocab_size
            for word in self.vocab:
                self.ngrams[context][word] = (self.ngrams[context].get(word, 0) + alpha)/total

    def prune_model(self, min_count=2):
        for context in list(self.ngrams.keys()):
            self.ngrams[context] = {
                k: v for k, v in self.ngrams[context].items() 
                if v >= min_count
            }
            if not self.ngrams[context]:
                del self.ngrams[context]
        def perplexity(self, test_corpus):
            total_log_prob = 0
            word_count = 0
            
            for sentence in test_corpus:
                tokens = preprocess(sentence)
                if len(tokens) < self.n:
                    continue
                    
                for i in range(self.n-1, len(tokens)):
                    context = tuple(tokens[i-self.n+1:i])
                    word = tokens[i]
                    
                    # Handle unseen contexts
                    prob = self.ngrams[context].get(word, 1e-10)  # Smoothing
                    total_log_prob += math.log(prob)
                    word_count += 1
                    
            if word_count == 0:
                return float('inf')  # Handle empty test case
            
            return math.exp(-total_log_prob / word_count)

    #enhanced prediction with backoff
    def predict(self, text, top_k=3):
        tokens = preprocess(text)
        print(f"Input tokens: {tokens}")  # Debug

        # Handle context window
        context = tuple(tokens[-(self.n-1):]) if len(tokens) >= self.n-1 else tuple(tokens)
        print(f"Initial context: {context}")  # Debug


        #backoff mechanism
        while len(context)>0:
            if context in self.ngrams:
                suggestions = sorted(self.ngrams[context].items(),key=lambda x: -x[1])[:top_k]
                return [word for word, prob in suggestions]
            context = context[1:] if len(context) > 1 else tuple()
        print(f"Backing off to: {context}")  # Debug
        #remove oldest word for backoff
            
        
        #final fallback to unigrams (most common words)
# Final fallback to most common words
        common_words = sorted(self.vocab, 
                            key=lambda x: sum(m[x] for m in self.ngrams.values()),
                            reverse=True)[:top_k]
        return common_words

In [38]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

with open('textfile.txt', 'r', encoding='utf-8') as file:
    corpus = sent_tokenize(file.read())  # Split into proper sentences

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Shiva\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:


#initialize and train model
model = NGramModel(n=3)
model.train(corpus=corpus)

#test predictions
print(model.predict("I enjoy"))

print(model.predict("science"))

print(model.predict("unknown text"))

Training on 2355 sentences...
Processing sentence: ['the', 'tragedie', 'of', 'hamlet', 'by', 'william', 'shakespeare', '1599', 'actus', 'primus']
Added n-gram: ('the', 'tragedie') -> of
Added n-gram: ('tragedie', 'of') -> hamlet
Added n-gram: ('of', 'hamlet') -> by
Added n-gram: ('hamlet', 'by') -> william
Added n-gram: ('by', 'william') -> shakespeare
Added n-gram: ('william', 'shakespeare') -> 1599
Added n-gram: ('shakespeare', '1599') -> actus
Added n-gram: ('1599', 'actus') -> primus
Processing sentence: ['scoena', 'prima']
Processing sentence: ['enter', 'barnardo', 'and', 'francisco', 'two', 'centinels']
Added n-gram: ('enter', 'barnardo') -> and
Added n-gram: ('barnardo', 'and') -> francisco
Added n-gram: ('and', 'francisco') -> two
Added n-gram: ('francisco', 'two') -> centinels
Processing sentence: ['barnardo']
Processing sentence: ['whos', 'there']
Processing sentence: ['fran']
Processing sentence: ['nay', 'answer', 'me', 'stand', 'vnfold', 'your', 'selfe', 'bar']
Added n-gram

In [ ]:
# File: interface.py
def interactive_demo(model):
    while True:
        user_input = input("\nType your text (press 'q' to quit):\n")
        if user_input.lower() == 'q':
            break
            
        predictions = model.predict(user_input)
        print(f"Suggestions: {predictions}")

# Run the demo
if __name__ == "__main__":
    interactive_demo(model)

Input tokens: ['hello', 'how', 'are']
Initial context: ('how', 'are')
Suggestions: ['they', 'crownes', 'knowne']


save model


In [ ]:
import pickle
# Save
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Load
with open('model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

print(loaded_model.predict("I enjoy"))  # Should work now

AttributeError: Can't pickle local object 'NGramModel.__init__.<locals>.<lambda>'

Evaluation Metrics

In [ ]:
import math

def perplexity(self, test_corpus):
    total_log_prob = 0
    word_count = 0
    
    for sentence in test_corpus:
        tokens = preprocess(sentence)
        for i in range(self.n-1, len(tokens)):
            context = tuple(tokens[i-self.n+1:i])
            word = tokens[i]
            prob = self.ngrams[context].get(word, 1e-10)  # Avoid zero
            total_log_prob += math.log(prob)
            word_count += 1
            
    return math.exp(-total_log_prob / word_count)

In [ ]:
test_perplexity = perplexity(corpus)
print(f"Model perplexity: {test_perplexity}")

TypeError: perplexity() missing 1 required positional argument: 'test_corpus'